In [12]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

import unicodedata
import unicodeblock.blocks

import os

sorted(os.listdir('data/현대 국어 사용 빈도 조사 2'))

['구통계.txt',
 '구통계.xlsx',
 '어미통계.txt',
 '어미통계.xlt',
 '음절통계.txt',
 '음절통계.xlt',
 '일반어휘통계.txt',
 '일반어휘통계.xlsx',
 '자모통계.txt',
 '자모통계.xlt',
 '조사통계.txt',
 '조사통계.xlt',
 '현대 국어 사용 빈도 조사 2(2005).PDF']

In [2]:
base_dir = 'data/현대 국어 사용 빈도 조사 2'

In [6]:
pd.read_excel(os.path.join(base_dir, '구통계.xlsx')).sort_values('빈도', ascending=False)[:50]

,순위,빈도,항목,풀이
2338,1,183,산업 사회,産業社會
892,2,175,금융 기관,金融機關
1704,3,165,문학 작품,文學作品
895,4,147,금융 실명제,金融實名制
1113,5,126,노동 운동,勞動運動
1319,6,121,대중 매체,大衆媒體
2282,7,111,사회 복지,社會福祉
4408,8,107,정치 자금,政治資金
1005,9,104,기자 회견,記者會見
613,10,103,구석기 시대,舊石器時代


In [7]:
pd.read_excel(os.path.join(base_dir, '일반어휘통계.xlsx')).sort_values('빈도', ascending=False)[:50]

,순위,빈도,어휘,풀이,품사
54657,1,97499,이다,NaN,지
3079,2,50558,것01,NaN,의
77100,3,42900,하다01,NaN,동
57046,4,39290,있다01,NaN,보
57047,5,37028,있다01,NaN,형
18309,6,24583,되다01,어른이 ～,동
39350,7,22484,수02,방법,의
11711,8,21415,나03,자아,대
77101,9,20880,하다01,NaN,보
47001,10,19195,없다01,NaN,형


In [11]:
pd.read_excel(os.path.join(base_dir, '자모통계.xlt')).sort_values('빈도', ascending=False)[:20]

,순위,빈도,위치,자모
31,1,2105587,초성,ㅇ
46,2,1926007,중성,ㅏ
66,3,1355527,중성,ㅣ
6,4,1328090,종성,ㄴ
0,5,1171038,초성,ㄱ
64,6,1151976,중성,ㅡ
50,7,942819,중성,ㅓ
54,8,862073,중성,ㅗ
9,9,800785,초성,ㄷ
13,10,776891,종성,ㄹ


In [15]:
pd.read_excel(os.path.join(base_dir, '조사통계.xlt')).sort_values('빈도', ascending=True)[:20]

,순위,빈도,항목,범주
55,131,1,ㄹ,인
74,131,1,럴,목
233,131,1,헌테서,부
182,131,1,이다,부
118,131,1,서부터,조
184,131,1,이다,조
119,131,1,설랑,부
191,131,1,이라곤,조
75,131,1,레,부
61,131,1,라고,부


In [18]:
pd.read_excel(os.path.join(base_dir, '음절통계.xlt')).sort_values('빈도', ascending=True)[:20]

,순위,빈도,음절
1264,1044,1,욤
1291,1044,1,윅
1282,1044,1,웟
1274,1044,1,웂
188,1044,1,껭
190,1044,1,껸
1249,1044,1,왑
1248,1044,1,왎
1237,1044,1,옰
1236,1044,1,옯
